In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
from PIL import Image
from docx import Document
from docx.shared import Pt
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls

# Đường dẫn tới Tesseract nếu bạn sử dụng Windows
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Bước 1: Đọc ảnh và tiền xử lý
img = cv2.imread('ta1.png', cv2.IMREAD_GRAYSCALE)

# Áp dụng threshold để chuyển ảnh thành đen trắng
_, binary_img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)

# Bước 2: Tìm đường kẻ ngang và dọc
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))  # Kernel cho đường ngang
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))   # Kernel cho đường dọc

# Tìm các đường kẻ ngang và dọc
horizontal_lines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
vertical_lines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

# Bước 3: Kết hợp các đường kẻ ngang và dọc
table_area = cv2.add(horizontal_lines, vertical_lines)

# Bước 4: Tìm các contours (đường viền) để phân chia bảng
contours, hierarchy = cv2.findContours(table_area, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Lưu trữ các bounding box của contours
bounding_boxes = [cv2.boundingRect(c) for c in contours]

# Loại bỏ contour lớn nhất (toàn bộ bảng lớn)
max_contour = max(bounding_boxes, key=lambda b: b[2] * b[3])
bounding_boxes = [b for b in bounding_boxes if b != max_contour]

# Sắp xếp bounding box theo vị trí (y, x) để giữ đúng thứ tự bảng
bounding_boxes = sorted(bounding_boxes, key=lambda b: (b[1], b[0]))

# Bước 5: Thực hiện OCR cho từng cell và lưu kết quả vào danh sách
table_data = []
for idx, (x, y, w, h) in enumerate(bounding_boxes):
    if w > 50 and h > 20:  # Điều chỉnh ngưỡng để bỏ qua các vùng quá nhỏ
        cell_img = img[y:y+h, x:x+w]
        
        # OCR cho cell
        text = pytesseract.image_to_string(cell_img).strip()
        
        # Thêm nội dung vào danh sách dữ liệu bảng
        table_data.append((x, y, w, h, text))

# Bước 6: Tạo file Word và chèn nội dung bảng
doc = Document()

# Tạo bảng với số hàng và số cột ước tính
# Giả sử bảng có số hàng và cột dựa trên bounding_boxes đã được phát hiện
rows = len(set([y for _, y, _, _, _ in table_data]))  # Số hàng dựa trên giá trị y
cols = len(set([x for x, _, _, _, _ in table_data]))  # Số cột dựa trên giá trị x

# Tạo bảng trong file Word
table = doc.add_table(rows=rows, cols=cols)

# Chèn nội dung vào các ô của bảng
sorted_table_data = sorted(table_data, key=lambda t: (t[1], t[0]))  # Sắp xếp theo vị trí y (hàng), rồi đến x (cột)
row_idx, col_idx = 0, 0
prev_y = sorted_table_data[0][1]

for idx, (x, y, w, h, text) in enumerate(sorted_table_data):
    # Nếu y thay đổi (tức là sang hàng mới), tăng chỉ số hàng và đặt lại chỉ số cột
    if y != prev_y:
        row_idx += 1
        col_idx = 0
        prev_y = y
    
    # Chèn nội dung vào ô tương ứng
    cell = table.cell(row_idx, col_idx)
    cell.text = text
    
    # Thêm đường viền cho ô
    cell._element.get_or_add_tcPr().append(parse_xml(r'<w:tcBorders {}>'
                                                      r'<w:left w:val="single" w:sz="4" wspace="0" ws="0"/>'
                                                      r'<w:right w:val="single" w:sz="4" wspace="0" ws="0"/>'
                                                      r'<w:top w:val="single" w:sz="4" wspace="0" ws="0"/>'
                                                      r'<w:bottom w:val="single" w:sz="4" wspace="0" ws="0"/>'
                                                      r'</w:tcBorders>'.format(nsdecls('w'))))

    col_idx += 1

# Lưu tài liệu thành file .doc
doc.save('output_table_with_borders.docx')

print("Đã lưu nội dung bảng vào file 'output_table_with_borders.docx'.")

Đã lưu nội dung bảng vào file 'output_table_with_borders.docx'.
